In [ ]:
# -- coding: utf-8 --
"""SmartSDLC-AI Gradio App"""

# Install dependencies (Colab only)
!pip install transformers torch gradio accelerate bitsandbytes PyPDF2

# Imports
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import PyPDF2

# ✅ Fix: Correct constructor name (double underscores)
class SmartSDLC_AI:
    def __init__(self):
        self.model_name = "ibm-granite/granite-3.3-2b-instruct"
        self.tokenizer = None
        self.model = None
        self.pipeline = None
        self.load_model()

    def load_model(self):
        try:
            print("🔄 Loading AI model...")
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True,
            )
            self.pipeline = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                max_length=1024,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
            print("✅ AI model loaded.")
        except Exception as e:
            print(f"❌ Error: {e}")
            print("⚠ Falling back to DialoGPT-medium...")
            fallback_model = "microsoft/DialoGPT-medium"
            self.tokenizer = AutoTokenizer.from_pretrained(fallback_model)
            self.model = AutoModelForCausalLM.from_pretrained(fallback_model)
            self.pipeline = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                max_length=1024,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
            print("✅ Fallback model loaded.")

    def analyze_requirements(self, text):
        prompt = f"You are a software requirement analysis assistant. Analyze the following requirements and list key functionalities, ambiguities, and improvement suggestions:\n\n{text}\n\nResponse:"
        response = self.pipeline(prompt)
        result = response[0]['generated_text'].split("Response:")[-1].strip()
        return result

    def generate_code(self, description):
        prompt = f"You are a software code generation assistant. Based on the following description, generate Python code:\n\n{description}\n\nCode:"
        response = self.pipeline(prompt)
        result = response[0]['generated_text'].split("Code:")[-1].strip()
        return result

# 📑 PDF Text Extraction Function
def extract_text_from_pdf(file_obj):
    reader = PyPDF2.PdfReader(file_obj)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

# 🖥 Gradio Interface Builder
def create_gradio_interface():
    with gr.Blocks(title="SmartSDLC-AI") as app:
        gr.HTML("<h1 style='text-align:center;'>🛠 SmartSDLC-AI</h1><p style='text-align:center;'>AI-powered Requirement Analysis & Code Generation Assistant</p>")

        with gr.Tabs():
            # 📄 Requirement Analysis Tab
            with gr.Tab("📄 Requirement Analysis"):
                with gr.Row():
                    pdf_input = gr.File(label="Upload PDF Requirements")
                    text_input = gr.Textbox(label="Or Enter Requirements Prompt", lines=6)
                analyze_btn = gr.Button("Analyze Requirements")
                analysis_output = gr.Textbox(label="Requirement Analysis Result", lines=12)

            # 💻 Code Generation Tab
            with gr.Tab("💻 Code Generation"):
                code_desc_input = gr.Textbox(label="Describe the Functionality for Code Generation", lines=6)
                generate_code_btn = gr.Button("Generate Code")
                code_output = gr.Code(label="Generated Python Code", language="python")

        # Functionality
        def handle_analysis(pdf_file, prompt_text):
            if pdf_file:
                text = extract_text_from_pdf(pdf_file)
            elif prompt_text.strip():
                text = prompt_text
            else:
                return "❗ Please upload a PDF or enter requirement text."
            return smart_sdlc_ai.analyze_requirements(text)

        def handle_code_generation(desc):
            if not desc.strip():
                return "❗ Please enter a description for code generation."
            return smart_sdlc_ai.generate_code(desc)

        analyze_btn.click(fn=handle_analysis, inputs=[pdf_input, text_input], outputs=analysis_output)
        generate_code_btn.click(fn=handle_code_generation, inputs=code_desc_input, outputs=code_output)

        gr.HTML("<p style='text-align:center; color:gray;'>⚙ Powered by IBM Granite AI | SmartSDLC-AI for Modern Development</p>")

    return app

# ✅ Fix: Correct entry point check
if __name__ == "__main__":
    print("🚀 SmartSDLC-AI Initializing...")
    smart_sdlc_ai = SmartSDLC_AI()
    iface = create_gradio_interface()
    print("🌐 Launching SmartSDLC-AI with public link...")
    iface.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cpu


✅ AI model loaded.
🌐 Launching SmartSDLC-AI with public link...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://501ebf5d3c6cb14259.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
